In [1]:
import numpy as np

class StepwiseMADS:
    def __init__(self, x0, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta0
        self.delta_min = delta_min
        self.alpha = alpha
        self.gamma = gamma
        self.mesh_size = delta0
        self.iteration = 0
        self.pending_eval = None
        self.best_value = None  # Store the best evaluation value
    
    def get_next_candidate(self):
        if self.pending_eval is not None:
            raise RuntimeError("Previous evaluation result is still pending.")
        
        directions = np.eye(len(self.x))
        candidates = [self.x + self.mesh_size * d for d in directions] + [self.x - self.mesh_size * d for d in directions]
        
        candidates = [np.clip(c, self.bounds[:, 0], self.bounds[:, 1]) for c in candidates]
        self.pending_eval = candidates.copy()
        return candidates
    
    def update_with_result(self, evaluations):
        if self.pending_eval is None:
            raise RuntimeError("No pending evaluations to update with.")

        best_idx = np.argmin(evaluations)
        best_candidate = self.pending_eval[best_idx]
        best_value = evaluations[best_idx]

        
        if self.best_value is None or best_value < self.best_value:
            self.x = best_candidate
            self.best_value = best_value
            self.mesh_size *= self.alpha
        else:
            self.mesh_size *= self.gamma
        
        self.mesh_size = max(self.mesh_size, self.delta_min)
        self.pending_eval = None
        self.iteration += 1
    
    def get_current_solution(self):
        return self.x




## Stp on every call

In [8]:
import math

class MADS:
    def __init__(self, x0, initial_value, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.stp_MADS = StepwiseMADS(x0, bounds=bounds, delta0=delta0, delta_min=delta_min, alpha=alpha, gamma=gamma)
        self.candidates = self.stp_MADS.get_next_candidate()
        self.evaluations = []
        self.step_num = 0
        self.stp_MADS.best_value = initial_value

    def step(self, evaluation):
        self.evaluations.append(evaluation)
        if self.candidates == []:
            # new MADS iteration
            print("no candidates...")
            self.stp_MADS.update_with_result(self.evaluations)
            self.candidates = self.stp_MADS.get_next_candidate()
            
            self.evaluations = []
        else:
            print("pending candidatas...")
        self.step_num +=1
        return self.candidates.pop(0)

In [9]:
# Example Usage
def obj_func(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

bounds = np.array([[0, 1], [10, 20], [0, 3]])
mads = StepwiseMADS(x0=[0.9, 19, 0.1], bounds=bounds)

In [10]:
x0=[0.9, 19, 0.1]
initial_value = obj_func(x0)
bounds = np.array([[0, 1], [10, 20], [0, 3]])
ab = MADS(x0, initial_value, bounds)

In [11]:
x = x0

In [12]:
for _ in range(120): 
    evaluation = obj_func(x)
    print(f"step {_}, current x {x}")
    x = ab.step(evaluation)
    print(f"step {_}, {evaluation}, next x, {x}")
    # print("ab.stp_MADS.pending_eval", ab.stp_MADS.pending_eval)

step 0, current x [0.9, 19, 0.1]
pending candidatas...
step 0, 18.12, next x, [ 1.  19.   0.1]
step 1, current x [ 1.  19.   0.1]
pending candidatas...
step 1, 18.21, next x, [ 0.9 20.   0.1]
step 2, current x [ 0.9 20.   0.1]
pending candidatas...
step 2, 27.12, next x, [ 0.9 19.   1.1]
step 3, current x [ 0.9 19.   1.1]
pending candidatas...
step 3, 16.32, next x, [ 0.  19.   0.1]
step 4, current x [ 0.  19.   0.1]
pending candidatas...
step 4, 18.21, next x, [ 0.9 18.   0.1]
step 5, current x [ 0.9 18.   0.1]
pending candidatas...
step 5, 11.12, next x, [ 0.9 19.   0. ]
step 6, current x [ 0.9 19.   0. ]
no candidates...
step 6, 18.41, next x, [ 1. 19.  0.]
step 7, current x [ 1. 19.  0.]
pending candidatas...
step 7, 18.5, next x, [ 0.9 20.   0. ]
step 8, current x [ 0.9 20.   0. ]
pending candidatas...
step 8, 27.41, next x, [ 0.9 19.   2. ]
step 9, current x [ 0.9 19.   2. ]
pending candidatas...
step 9, 16.41, next x, [ 0. 19.  0.]
step 10, current x [ 0. 19.  0.]
pending candid